In [1]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import numpy as np
import yfinance as yf
from concurrent import futures
import matplotlib.pyplot as plt

from statsmodels.tsa.seasonal import seasonal_decompose

import quandl
#from fbprophet import Prophet

import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
sns.set()

In [2]:
os.chdir('../Statistics/')

In [3]:
from keras.models import load_model

In [4]:
nifty_bank_shares = pd.read_excel('nifty bank shares.xlsx',header=0)

In [5]:
os.chdir('../Portfolio/')

In [6]:
import datetime

In [7]:
end = str(datetime.date.today()+datetime.timedelta(days=1))
start = str(datetime.date.today()-datetime.timedelta(days=100))

In [8]:
end,start

('2021-01-30', '2020-10-21')

In [9]:
bad_tickers = []
def get_data(symbol):
    try:
        df = yf.download(symbol+'.NS',start=start,end=end)
        df['Symbol'] = symbol
        df.reset_index(inplace=True)
        df = df[['Date','Symbol','Open','High','Low','Close','Adj Close','Volume']]
        df = df.append(pd.DataFrame([[end,symbol,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]],columns=df.columns))
        df.to_csv(symbol+'.csv',index=False)
    except:
        bad_tickers.append(symbol)
        print(f'Data not found for {symbol}')

In [10]:
for symbol in list(nifty_bank_shares['Symbol']):
    get_data(symbol)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [11]:
df = yf.download('^NSEBANK',start=start,end=end)

[*********************100%***********************]  1 of 1 completed


In [12]:
df['Symbol'] = 'Nifty Bank'
df.reset_index(inplace=True)
df = df[['Date','Symbol','Open','High','Low','Close','Adj Close','Volume']]
df = df.append(pd.DataFrame([[end,symbol,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]],columns=df.columns))
df.to_csv('niftybank.csv')

In [13]:
niftybank = pd.read_csv('niftybank.csv',header=0)

In [14]:
import talib

In [15]:
#create nifty variables
nifty = pd.read_csv('niftybank.csv',header=0,parse_dates=True)
nifty['Date'] = nifty['Date'].apply(lambda x: str.strip(str(x)))
nifty['returns'] = np.log(nifty['Adj Close']/nifty['Adj Close'].shift(1))
for i in range(1,11):
    nifty[f'return_{i}d'] = nifty['returns'].shift(i)
nifty['MA5'] = nifty['Adj Close'].shift(1).rolling(window=5).mean()
nifty['MA20'] = nifty['Adj Close'].shift(1).rolling(window=20).mean()
nifty['MA5_ratio'] = nifty['Adj Close'].shift(1)/nifty['MA5']
nifty['MA20_ratio'] = nifty['Adj Close'].shift(1)/nifty['MA20']
nifty['High5'] = nifty['Adj Close'].shift(1).rolling(window=5).max()
nifty['Low5'] = nifty['Adj Close'].shift(1).rolling(window=5).min()
nifty['High20'] = nifty['Adj Close'].shift(1).rolling(window=20).max()
nifty['Low20'] = nifty['Adj Close'].shift(1).rolling(window=20).min()
nifty['High5_ratio'] = nifty['Adj Close'].shift(1)/nifty['High5']
nifty['Low5_ratio'] = nifty['Low5']/nifty['Adj Close'].shift(1)
nifty['High20_ratio'] = nifty['Adj Close'].shift(1)/nifty['High20']
nifty['Low20_ratio'] = nifty['Low20']/nifty['Adj Close'].shift(1)
nifty['Voltality5'] = nifty['returns'].shift(1).rolling(window=5).std()
nifty['Voltality20'] = nifty['returns'].shift(1).rolling(window=20).std()

In [16]:
nifty.shape

(68, 34)

In [17]:
nifty.tail(2)

,Unnamed: 0,Date,Symbol,Open,High,Low,Close,Adj Close,Volume,returns,...,High5,Low5,High20,Low20,High5_ratio,Low5_ratio,High20_ratio,Low20_ratio,Voltality5,Voltality20
66,66,2021-01-29 00:00:00,Nifty Bank,30700.650391,31111.949219,30254.550781,30565.5,30565.5,0.0,0.006802,...,32186.900391,30284.550781,32574.650391,30284.550781,0.943188,0.997571,0.931961,0.997571,0.016448,0.012222
67,0,2021-01-30,PNB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,31198.400391,30284.550781,32574.650391,30284.550781,0.979714,0.990808,0.938322,0.990808,0.018967,0.011849


In [18]:
def create_variables(symbol):
    df = pd.read_csv(symbol+'.csv',header=0)
    df['Date'] = df['Date'].apply(lambda x:str.strip(str(x)))
    df['returns'] = np.log(df['Adj Close']/df['Adj Close'].shift(1))
    for i in range(1,11):
        df[f'return_{i}d'] = df['returns'].shift(i)
    df['Open_Chg'] = np.log(df['Open'].shift(1)/df['Open'].shift(2))
    df['High_Chg'] = np.log(df['High'].shift(1)/df['High'].shift(2))
    df['Low_Chg'] = np.log(df['Low'].shift(1)/df['Low'].shift(2))
    df['Range_Chg'] = np.log((df['High'].shift(1)-df['Low'].shift(1))/(df['High'].shift(2)-df['Low'].shift(2)))
    df['Volume_Chg'] = np.log(df['Volume'].shift(1)/df['Volume'].shift(2))
    df['Voltality5'] = df['returns'].shift(1).rolling(window=5).std()
    df['Voltality20'] = df['returns'].shift(1).rolling(window=20).std()
    df['MA5'] = df['Adj Close'].shift(1).rolling(window=5).mean()
    df['MA20'] = df['Adj Close'].shift(1).rolling(window=20).mean()
    df['High5'] = df['Adj Close'].shift(1).rolling(window=5).max()
    df['Low5'] = df['Adj Close'].shift(1).rolling(window=5).min()
    df['High20'] = df['Adj Close'].shift(1).rolling(window=20).max()
    df['Low20'] = df['Adj Close'].shift(1).rolling(window=20).min()
    df['MA5_ratio'] = df['Adj Close'].shift(1)/df['MA5']
    df['MA20_ratio'] = df['Adj Close'].shift(1)/df['MA20']
    df['High5_ratio'] = df['Adj Close'].shift(1)/df['High5']
    df['Low5_ratio'] = df['Low5']/df['Adj Close'].shift(1)
    df['High20_ratio'] = df['Adj Close'].shift(1)/df['High20']
    df['Low20_ratio'] = df['Low20']/df['Adj Close'].shift(1)
    df['future_return'] = np.log(df['Adj Close'].shift(-1)/df['Adj Close'])
    df['lift'] = df['future_return'].apply(lambda x: 1 if x >0 else 0)
    df['Price_Voltality5'] = df['Adj Close'].shift(1).rolling(window=5).std()
    df['Price_Voltality20'] = df['Adj Close'].shift(1).rolling(window=20).std()
    df['Bollinger5_ratio'] = (df['Adj Close'].shift(1)-df['MA5']+2*df['Price_Voltality5'])/(4*df['Price_Voltality5'])
    df['Bollinger20_ratio'] = (df['Adj Close'].shift(1)-df['MA20']+2*df['Price_Voltality20'])/(4*df['Price_Voltality20'])
    df['ADX5'] = talib.ADX(df['High'].shift(1),df['Low'].shift(1),df['Close'].shift(1),5)/100
    df['ADX20'] = talib.ADX(df['High'].shift(1),df['Low'].shift(1),df['Close'].shift(1),20)/100
    df['RSI5'] = talib.RSI(df['Adj Close'].shift(1),5)/100
    df['RSI20'] = talib.RSI(df['Adj Close'].shift(1),20)/100
    df = df.merge(nifty,left_on='Date',right_on='Date',suffixes=['','_nifty'],how='inner')
    df['correlation'] = df.apply(lambda x : np.corrcoef(list(x[[f'return_{i}d' for i in range(1,11)]]),list(x[[f'return_{i}d_nifty' for i in range(1,11)]]))[0,1],axis=1)
    return df

In [19]:
df = pd.DataFrame()
for symbol in list(nifty_bank_shares['Symbol']):
    print(symbol)
    df = df.append(create_variables(symbol))

HDFCBANK
ICICIBANK
KOTAKBANK
AXISBANK
SBIN
INDUSINDBK
BANDHANBNK
FEDERALBNK
RBLBANK
IDFCFIRSTB
BANKBARODA
PNB


In [20]:
dates = sorted(df['Date'].unique())

In [21]:
dates[-1]

'2021-01-30'

In [22]:
dates = dates[-3:]

In [23]:
df = df[df['Date'].isin(dates)]

In [24]:
df_test = df[df['Date']==dates[2]]

In [25]:
for col in df_test.columns:
    if df_test[col].isnull().any():
        print(col)

Open
High
Low
Close
Adj Close
Volume
returns
future_return
Open_nifty
High_nifty
Low_nifty
Close_nifty
Adj Close_nifty
Volume_nifty
returns_nifty


In [26]:
prob_model = load_model('../Models/prod_prob_model.h5')
pos_return_model = load_model('../Models/prod_pos_return_model.h5')
neg_return_model = load_model('../Models/prod_neg_return_model.h5')

In [27]:
header = pd.read_excel('../Input_Data/header.xlsx',header=0)

In [28]:
prob_inputs = [col for col in list(header.columns) if header.loc[1,col]=='N']
pos_return_inputs = [col for col in list(header.columns) if header.loc[2,col]=='N']
neg_return_inputs = [col for col in list(header.columns) if header.loc[3,col]=='N']

In [29]:
#get equation
def get_equation(model):
    mat1 = np.matrix(model.layers[1].get_weights()[0])
    inputs = mat1.shape[0]
    mat2 = np.matrix(model.layers[1].get_weights()[1])
    mat3 = np.concatenate((mat1,mat2),axis=0)
    mat4 = np.matrix(model.layers[2].get_weights()[0])
    mat5 = np.matmul(mat3,mat4)
    mat5[inputs,:]+=np.matrix(model.layers[2].get_weights()[1])
    mat6 = np.matrix(model.layers[3].get_weights()[0])
    mat7 = np.matmul(mat5,mat6)
    mat7[inputs,:]+=np.matrix(model.layers[3].get_weights()[1])
    mat8 = np.matrix(model.layers[4].get_weights()[0])
    mat9 = np.matmul(mat7,mat8)
    mat9[inputs,:]+=np.matrix(model.layers[4].get_weights()[1])
    
    return [float(x) for x in mat9[:,0]]

In [30]:
eq_prob = pd.read_excel('../Statistics/dynamic_prob_eqn.xlsx',header=0)

In [31]:
eq_pos_returns = pd.read_excel('../Statistics/dynamic_pos_return_eqn.xlsx',header=0)

In [32]:
eq_neg_returns = pd.read_excel('../Statistics/dynamic_neg_return_eqn.xlsx')

In [33]:
dates[0]

'2021-01-28 00:00:00'

In [34]:
df.loc[df['Date']==dates[0],['future_return','lift']]

,future_return,lift
65,0.013795,1
65,0.016428,1
65,-0.005763,0
65,-0.011698,0
65,-0.000886,0
65,0.052781,1
65,-0.005479,0
65,0.009729,1
65,-0.005830,0
65,-0.014722,0


In [35]:
prob_model.fit(df.loc[df['Date']==dates[0],prob_inputs],df.loc[df['Date']==dates[0],'lift'],epochs=1000)
pos_data = df.loc[(df['Date']==dates[0]) & (df['future_return']>0),:]
neg_data = df.loc[(df['Date']==dates[0]) & (df['future_return']<=0),:]
if pos_data.shape[0] > 0:
    pos_return_model.fit(pos_data[pos_return_inputs],pos_data['future_return'],epochs=1000)
if neg_data.shape[0] > 0:
    neg_return_model.fit(neg_data[neg_return_inputs],neg_data['future_return'],epochs=1000)

eq_prob = eq_prob.append(pd.DataFrame([[dates[0]]+get_equation(prob_model)],columns=eq_prob.columns))
eq_pos_returns = eq_pos_returns.append(pd.DataFrame([[dates[0]]+get_equation(pos_return_model)],columns=eq_pos_returns.columns))
eq_neg_returns = eq_neg_returns.append(pd.DataFrame([[dates[0]]+get_equation(neg_return_model)],columns=eq_neg_returns.columns))

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 0.7075
Epoch 2/1000
1/1 [==============================] - 0s 11ms/step - loss: 0.7075
Epoch 3/1000
1/1 [==============================] - 0s 11ms/step - loss: 0.7075
Epoch 4/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.7075
Epoch 5/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.7075
Epoch 6/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.7075
Epoch 7/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.7075
Epoch 8/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.7075
Epoch 9/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.7075
Epoch 10/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.7075
Epoch 11/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.7075
Epoch 12/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.7075
Epoch 13/1000
1/1 [=============================

1/1 [==============================] - 0s 5ms/step - loss: 0.7075
Epoch 104/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.7075
Epoch 105/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.7075
Epoch 106/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.7075
Epoch 107/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.7075
Epoch 108/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.7075
Epoch 109/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.7075
Epoch 110/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.7075
Epoch 111/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.7075
Epoch 112/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.7075
Epoch 113/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.7075
Epoch 114/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.7075
Epoch 115/1000
1/1 [=======================

1/1 [==============================] - 0s 5ms/step - loss: 0.7074
Epoch 204/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.7074
Epoch 205/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.7074
Epoch 206/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.7074
Epoch 207/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.7074
Epoch 208/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.7074
Epoch 209/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.7074
Epoch 210/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.7074
Epoch 211/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.7074
Epoch 212/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.7074
Epoch 213/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.7074
Epoch 214/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.7074
Epoch 215/1000
1/1 [=======================

1/1 [==============================] - 0s 5ms/step - loss: 0.7073
Epoch 305/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.7073
Epoch 306/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.7073
Epoch 307/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.7073
Epoch 308/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.7073
Epoch 309/1000
1/1 [==============================] - ETA: 0s - loss: 0.707 - 0s 5ms/step - loss: 0.7073
Epoch 310/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.7073
Epoch 311/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.7073
Epoch 312/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.7073
Epoch 313/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.7073
Epoch 314/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.7073
Epoch 315/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.7073
Epoch 316/1000
1/1 

1/1 [==============================] - 0s 4ms/step - loss: 0.7073
Epoch 406/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.7073
Epoch 407/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.7073
Epoch 408/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.7073
Epoch 409/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.7073
Epoch 410/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.7073
Epoch 411/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.7073
Epoch 412/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.7073
Epoch 413/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.7073
Epoch 414/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.7073
Epoch 415/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.7073
Epoch 416/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.7073
Epoch 417/1000
1/1 [=======================

...

1/1 [==============================] - 0s 5ms/step - loss: 1.6079e-05
Epoch 543/1000
1/1 [==============================] - 0s 4ms/step - loss: 1.6077e-05
Epoch 544/1000
1/1 [==============================] - 0s 10ms/step - loss: 1.6076e-05
Epoch 545/1000
1/1 [==============================] - 0s 6ms/step - loss: 1.6074e-05
Epoch 546/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.6073e-05
Epoch 547/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.6072e-05
Epoch 548/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.6070e-05
Epoch 549/1000
1/1 [==============================] - 0s 4ms/step - loss: 1.6069e-05
Epoch 550/1000
1/1 [==============================] - 0s 4ms/step - loss: 1.6067e-05
Epoch 551/1000
1/1 [==============================] - 0s 4ms/step - loss: 1.6066e-05
Epoch 552/1000
1/1 [==============================] - 0s 4ms/step - loss: 1.6065e-05
Epoch 553/1000
1/1 [==============================] - 0s 4ms/step - loss: 1.606

1/1 [==============================] - 0s 4ms/step - loss: 1.5943e-05
Epoch 639/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.5941e-05
Epoch 640/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.5939e-05
Epoch 641/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.5938e-05
Epoch 642/1000
1/1 [==============================] - 0s 4ms/step - loss: 1.5937e-05
Epoch 643/1000
1/1 [==============================] - 0s 4ms/step - loss: 1.5935e-05
Epoch 644/1000
1/1 [==============================] - 0s 6ms/step - loss: 1.5934e-05
Epoch 645/1000
1/1 [==============================] - 0s 4ms/step - loss: 1.5932e-05
Epoch 646/1000
1/1 [==============================] - 0s 4ms/step - loss: 1.5931e-05
Epoch 647/1000
1/1 [==============================] - 0s 4ms/step - loss: 1.5929e-05
Epoch 648/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.5928e-05
Epoch 649/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.5927

1/1 [==============================] - 0s 4ms/step - loss: 1.5801e-05
Epoch 735/1000
1/1 [==============================] - 0s 4ms/step - loss: 1.5799e-05
Epoch 736/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.5798e-05
Epoch 737/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.5796e-05
Epoch 738/1000
1/1 [==============================] - 0s 4ms/step - loss: 1.5795e-05
Epoch 739/1000
1/1 [==============================] - 0s 4ms/step - loss: 1.5793e-05
Epoch 740/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.5792e-05
Epoch 741/1000
1/1 [==============================] - 0s 4ms/step - loss: 1.5790e-05
Epoch 742/1000
1/1 [==============================] - 0s 4ms/step - loss: 1.5789e-05
Epoch 743/1000
1/1 [==============================] - 0s 6ms/step - loss: 1.5787e-05
Epoch 744/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.5786e-05
Epoch 745/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.5784

1/1 [==============================] - 0s 3ms/step - loss: 1.5655e-05
Epoch 831/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.5654e-05
Epoch 832/1000
1/1 [==============================] - 0s 4ms/step - loss: 1.5652e-05
Epoch 833/1000
1/1 [==============================] - 0s 4ms/step - loss: 1.5651e-05
Epoch 834/1000
1/1 [==============================] - 0s 4ms/step - loss: 1.5649e-05
Epoch 835/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.5647e-05
Epoch 836/1000
1/1 [==============================] - 0s 4ms/step - loss: 1.5646e-05
Epoch 837/1000
1/1 [==============================] - 0s 4ms/step - loss: 1.5644e-05
Epoch 838/1000
1/1 [==============================] - 0s 3ms/step - loss: 1.5643e-05
Epoch 839/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.5641e-05
Epoch 840/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.5640e-05
Epoch 841/1000
1/1 [==============================] - 0s 4ms/step - loss: 1.5638

1/1 [==============================] - 0s 6ms/step - loss: 1.5506e-05
Epoch 926/1000
1/1 [==============================] - 0s 4ms/step - loss: 1.5504e-05
Epoch 927/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.5502e-05
Epoch 928/1000
1/1 [==============================] - 0s 4ms/step - loss: 1.5501e-05
Epoch 929/1000
1/1 [==============================] - 0s 4ms/step - loss: 1.5499e-05
Epoch 930/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.5498e-05
Epoch 931/1000
1/1 [==============================] - 0s 6ms/step - loss: 1.5496e-05
Epoch 932/1000
1/1 [==============================] - 0s 4ms/step - loss: 1.5494e-05
Epoch 933/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.5493e-05
Epoch 934/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.5491e-05
Epoch 935/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.5489e-05
Epoch 936/1000
1/1 [==============================] - 0s 6ms/step - loss: 1.5488

In [36]:
prob_model.save('../Models/prod_prob_model.h5')
pos_return_model.save('../Models/prod_pos_return_model.h5')
neg_return_model.save('../Models/prod_neg_return_model.h5')

In [37]:
eq_prob.to_excel('../Statistics/dynamic_prob_eqn.xlsx',index=False)
eq_pos_returns.to_excel('../Statistics/dynamic_pos_return_eqn.xlsx',index=False)
eq_neg_returns.to_excel('../Statistics/dynamic_neg_return_eqn.xlsx',index=False)

In [38]:
for col in df_test.columns:
    if df_test[col].isnull().any():
        print(col)

Open
High
Low
Close
Adj Close
Volume
returns
future_return
Open_nifty
High_nifty
Low_nifty
Close_nifty
Adj Close_nifty
Volume_nifty
returns_nifty


In [39]:
df_test['expected_probability'] = prob_model.predict(df_test[prob_inputs])
df_test['expected_positive_return'] = pos_return_model.predict(df_test[pos_return_inputs])
df_test['expected_negative_return'] = neg_return_model.predict(df_test[neg_return_inputs])

In [40]:
df_test.to_excel('data_predictions.xlsx',index=False)

In [41]:
df_prices = df[df['Date']==dates[1]][['Symbol','Close','Adj Close']]

In [42]:
df_prices.to_excel('prices.xlsx',index=False)